In [4]:
#Code to create 3 150*150 images by taking optical flow from 2 alternate frames
import os
import numpy as np
import cv2 as cv
import random
import matplotlib as plt
from cv2.optflow import calcOpticalFlowSF
from PIL import Image

In [5]:
#Function to generate an image array when centre pixels are provided
def create_image(frame,i,j):
    if i-75>=0:
        if i+75<=1080-1:
            if j-75>=0:
                if j+75<=1920-1:
                    new_image=frame[i-75:i+75,j-75:j+75,0:3]
                else:
                    new_image=frame[i-75:i+75,1920-1-150:1920-1,0:3]
            else:
                new_image=frame[i-75:i+75,0:150,0:3]
        elif j-75>=0:
                if j+75<=1920-1:
                    new_image=frame[1080-1-150:1080-1,j-75:j+75,0:3]
                else:
                    new_image=frame[1080-1-150:1080-1,1920-1-150:1920-1,0:3]
        else:
            new_image=frame[1080-1-150:1080-1,0:150,0:3]
    elif j-75>=0:
                if j+75<=1920-1:
                    new_image=frame[0:150,j-75:j+75,0:3]
                else:
                    new_image=frame[0:150,1920-1-150:1920-1,0:3]
    else:
        new_image=frame[0:150,0:150,0:3]
    return new_image

#Function to calculate average value optical flow in two images and return indexes of pixels having max individual flow value
def calc_simple_flow(image1,image2):
    flow = calcOpticalFlowSF(image1, image2, layers=3, averaging_block_size=3, max_flow=4)
    n = np.sum(1 - np.isnan(flow), axis=0)
    n = np.sum(n,axis=0)
    # print(n)
    flow[np.isnan(flow)] = 0
    return np.linalg.norm(np.sum(flow, axis=(0, 1)) / n),np.unravel_index(flow.argmax(), flow.shape)[0:2],np.unravel_index(flow.argmin(), flow.shape)[0:2]

#Function to generate pixel using max flow and min flow found randomly
def create_random_image_crops_pixels(frame1,frame2,random_number=10):
    max=0
    min=256
    for x in range(random_number):
        i = random.randint(0,1080)
        j=random.randint(0,1920)
        temp_image1=create_image(frame1,i,j)
        temp_image2=create_image(frame2,i,j)
        temp_flow,_,_=calc_simple_flow(temp_image1,temp_image2)
#         print(temp_flow)
        if temp_flow>max:
            i1=i
            j1=j
            max=temp_flow
        if temp_flow<min:
            i2=i
            j2=j
            min=temp_flow
#     print(i,j,max)
    return (i1,j1),(i2,j2)
#(i1,j1) has pixel values for max flow and (i2,j2) for least


#Function to generate individual pixel for which flow is max or min
def create_image_crops_pixels(frame1,frame2):
    _,index_high,index_low=calc_simple_flow(frame1,frame2)
    return index_high,index_low

def save_image(frames,pixel_values,type="train",example_number=0):
    image=create_image(frames[0],pixel_values[0],pixel_values[1])
    cv.imwrite("./"+type+"_dataset/"+type+"_"+str(example_number)+"_0.jpg",image)
    image=create_image(frames[1],pixel_values[0],pixel_values[1])
    cv.imwrite("./"+type+"_dataset/"+type+"_"+str(example_number)+"_1.jpg",image)
    image=create_image(frames[2],pixel_values[0],pixel_values[1])
    cv.imwrite("./"+type+"_dataset/"+type+"_"+str(example_number)+"_2.jpg",image)
    return

In [6]:
example_number=0

# Read RGB image 
frame0 = cv.imread("00000.jpg")  
frame2 = cv.imread("00002.jpg")  
frame1 = cv.imread("00001.jpg")
frames=(frame0,frame1,frame2)

In [7]:
pixel_values1,pixel_values2=create_random_image_crops_pixels(frame0,frame2)
save_image(frames,pixel_values1,"train",example_number)
example_number+=1
save_image(frames,pixel_values2,"train",example_number)
example_number+=1




In [ ]:


pixel_values1,pixel_values2=create_random_image_crops_pixels(frame0,frame2,100)
save_image(frames,pixel_values1,"train",example_number)
example_number+=1
save_image(frames,pixel_values2,"train",example_number)
example_number+=1


index_high,index_low=create_image_crops_pixels(frame0,frame2)
save_image(frames,index_high,"train",example_number)
example_number+=1
save_image(frames,index_low,"train",example_number)
example_number+=1
